In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Sat Sep 15 2018 

CPython 3.6.6
IPython 6.5.0

numpy 1.15.1
scipy 1.1.0
sklearn 0.19.1
pandas 0.23.4

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.15.0-33-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [44]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import os
import mne

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
DATA_ROOT = os.path.abspath(os.path.join(PROJ_ROOT, 'data'))
print(PROJ_ROOT)
print(DATA_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
from data.preprocess import read_raw_data, preprocess_data

/home/miroslav/Source/thesis_project
/home/miroslav/Source/thesis_project/data


In [25]:
column_names = ['freq', 'RESP_4W', 'RESP_FIN', 'REMISE_FIN', 'AGE', 'SEX', 'M_1',
       'M_4', 'M_F', 'délka léčby', 'lék 1', 'lék 2', 'lék 3', 'lék 4']
raw_root = os.path.abspath(os.path.join(DATA_ROOT, 'raw'))
filename = 'DEP-POOL_Final_144.xlsx'
meta_df = pd.read_excel(os.path.join(raw_root, filename), index_col='ID', names=column_names)

In [30]:
meta_df.iloc[0]['freq']

250

In [65]:
CHANNEL_NAMES = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']

def read_raw(file_name):
    df = pd.read_table(file_name, sep='\t', names=CHANNEL_NAMES, skiprows=[0])
    return df
    
def read_mne_raw_data(file_path):
    assert(file_path.endswith('.tdt'))
    no_ext_file_name = os.path.splitext(file_path.split(os.sep)[-1])[0] 
    trial_num = no_ext_file_name[-1]
    index = int(no_ext_file_name[:-1])
    sampling_freq = meta_df.iloc[index-1]['freq']
    
    df = read_raw(file_path)
    info = mne.create_info(ch_names=CHANNEL_NAMES, sfreq=sampling_freq,
                           ch_types = 'eeg')
    data = mne.io.RawArray(np.transpose(df.values), info)
    return data

In [69]:
def preprocess_raw():
    for file_name in os.listdir(raw_root):
        if not file_name.endswith('.tdt'):
            print('Skipping file %s' % file_name)
            continue
        file_path = os.path.join(raw_root, file_name)
        mne_raw_data = read_mne_raw_data(file_path)
        
        # Remove power line noise, may not be necessary depending on the data
        mne_raw_data.notch_filter(np.arange(50, 125, 50), filter_length='auto', phase='zero')
        
        # Or we can simply low pass filter, which may be better
        # mne_raw_data.filter(None, 50., fir_design='firwin')
        
        # Remove slow drifts via high pass, bad practice in some situations, but may improve ICA
        # mne_raw_data.filter(1., None, fir_design='firwin')
        
        # Downsample high frequency recordings to reduce the amount of data
        if int(mne_raw_data.info['sfreq']) == 1000:
            print('Downsampling data for file %s' % file_name)
            mne_raw_data.resample(250, npad="auto")
        
preprocess_raw()

Creating RawArray with float64 data, n_channels=19, n_times=18068
    Range : 0 ... 18067 =      0.000 ...    72.268 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=15331
    Range : 0 ... 15330 =      0.000 ...    61.320 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=61293
    Range : 0 ... 61292 =      0.000 ...    61.292 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file 113a.tdt
Creating RawArray with float64 data, n_channels=19, n_times=23752
    Range : 0 ... 23751 =      0.000 ...    95.004 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=17602
    Range : 0 ... 17601 =      0.000 ...    70.404 secs
Ready.
Set

Creating RawArray with float64 data, n_channels=19, n_times=85278
    Range : 0 ... 85277 =      0.000 ...    85.277 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file 11b.tdt
Creating RawArray with float64 data, n_channels=19, n_times=17062
    Range : 0 ... 17061 =      0.000 ...    68.244 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=12932
    Range : 0 ... 12931 =      0.000 ...    51.724 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=15509
    Range : 0 ... 15508 =      0.000 ...    62.032 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=21893
    Range : 0 ... 21892 =      0.000 ...    87.568 secs
Ready.
Sett

    Range : 0 ... 20654 =      0.000 ...    82.616 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=18680
    Range : 0 ... 18679 =      0.000 ...    74.716 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=15009
    Range : 0 ... 15008 =      0.000 ...    60.032 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=15623
    Range : 0 ... 15622 =      0.000 ...    62.488 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=60942
    Range : 0 ... 60941 =      0.000 ...    60.941 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file

Creating RawArray with float64 data, n_channels=19, n_times=25509
    Range : 0 ... 25508 =      0.000 ...   102.032 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=77491
    Range : 0 ... 77490 =      0.000 ...    77.490 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file 58a.tdt
Creating RawArray with float64 data, n_channels=19, n_times=82243
    Range : 0 ... 82242 =      0.000 ...    82.242 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file 18a.tdt
Creating RawArray with float64 data, n_channels=19, n_times=17194
    Range : 0 ... 17193 =      0.000 ...    68.772 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=7656
    Range : 0 ... 7655 =      0.0

Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=7744
    Range : 0 ... 7743 =      0.000 ...    30.972 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=22942
    Range : 0 ... 22941 =      0.000 ...    91.764 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=28997
    Range : 0 ... 28996 =      0.000 ...   115.984 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=110376
    Range : 0 ... 110375 =      0.000 ...   110.375 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file 82a.tdt
Creating RawArray with float64 data, n_channels=19, n_times=16279
    Range : 0 ..

    Range : 0 ... 61968 =      0.000 ...    61.968 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file 80b.tdt
Creating RawArray with float64 data, n_channels=19, n_times=21507
    Range : 0 ... 21506 =      0.000 ...    86.024 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=62047
    Range : 0 ... 62046 =      0.000 ...    62.046 secs
Ready.
Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected
Downsampling data for file 56a.tdt
Creating RawArray with float64 data, n_channels=19, n_times=15545
    Range : 0 ... 15544 =      0.000 ...    62.176 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=17765
    Range : 0 ... 17764 =      0.000 ...    71.056 secs
Ready.
Setting up band-stop filter
Filter 

Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=23526
    Range : 0 ... 23525 =      0.000 ...    94.100 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=19189
    Range : 0 ... 19188 =      0.000 ...    76.752 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=18893
    Range : 0 ... 18892 =      0.000 ...    75.568 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=17065
    Range : 0 ... 17064 =      0.000 ...    68.256 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=84529
    Range : 0 ..

Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
Creating RawArray with float64 data, n_channels=19, n_times=21861
    Range : 0 ... 21860 =      0.000 ...    87.440 secs
Ready.
Setting up band-stop filter
Filter length of 1651 samples (6.604 sec) selected
